In [5]:
import pandas as pd
import networkx as nx
import numpy as np

edges = pd.read_csv('hero-network.csv')  

G = nx.from_pandas_edgelist(
    edges,
    source='hero1',
    target='hero2',
    create_using=nx.Graph()
)

num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

deg_cent = nx.degree_centrality(G)
cent_values = np.array(list(deg_cent.values()))

avg_cent = cent_values.mean()
std_cent = cent_values.std(ddof=0)
min_cent = cent_values.min()
max_cent = cent_values.max()

node_max = max(deg_cent, key=deg_cent.get)
node_min = min(deg_cent, key=deg_cent.get)
nodes_near_avg = sorted(
    deg_cent,
    key=lambda n: abs(deg_cent[n] - avg_cent)
)[:5]

print(f"Total nodes:               {num_nodes}")
print(f"Total edges:               {num_edges}")
print(f"Avg degree centrality:     {avg_cent:.6f}")
print(f"Standard dev centrality:   {std_cent:.6f}")
print(f"Min centrality:            {min_cent:.6f}")
print(f"Max centrality:            {max_cent:.6f}")
print()
print(f"Node with max centrality:  {node_max} ({deg_cent[node_max]:.6f})")
print(f"Node with min centrality:  {node_min} ({deg_cent[node_min]:.6f})")
print("Top 5 nodes closest to avg centrality:")
for n in nodes_near_avg:
    print(f"  {n} ({deg_cent[n]:.6f})")


Total nodes:               6426
Total edges:               167219
Avg degree centrality:     0.008100
Standard dev centrality:   0.017689
Min centrality:            0.000156
Max centrality:            0.296965

Node with max centrality:  CAPTAIN AMERICA (0.296965)
Node with min centrality:  AZRAEL (0.000156)
Top 5 nodes closest to avg centrality:
  JONES, TAMMY ANNE (0.008093)
  JONES, LORRAINE LORR (0.008093)
  JONES, DANIEL DANNY (0.008093)
  NEVILLE, KATE (0.008093)
  PEARSON, MARCY (0.008093)
